In [1]:
import os
import time


In [2]:
#Size of the file
os.path.getsize('/content/Rate.csv')

428867584

In [3]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('/content/Rate.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.01694655418395996 sec


In [4]:
import pandas as pd
start = time.time()
df = pd.read_csv('/content/Rate.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  15.812951564788818 sec


Here Dask is better than pandas 

In [5]:
from dask import dataframe as dd
df = dd.read_csv('/content/Rate.csv',delimiter=',')

In [6]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 24 entries, BusinessYear to RowNumber
dtypes: object(10), float64(9), int64(5)

In [7]:
#No. of Rows
len(df.index)

2834695

In [8]:
#No, of Columns
len(df.columns)

24

In [9]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

<ipython-input-9-554f595a0ef5>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns=df.columns.str.replace('[#,@,&]','')


In [10]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [11]:
data=df.columns
data

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'FederalTIN', 'RateEffectiveDate',
       'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age',
       'IndividualRate', 'IndividualTobaccoRate', 'Couple',
       'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents',
       'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent',
       'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents',
       'RowNumber'],
      dtype='object')

validation

In [12]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [13]:
%%writefile utility.py
import yaml
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [14]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - BusinessYear
      - StateCode
      - IssuerId
      - SourceName
      - VersionNum
      - ImportDate
      - IssuerId2
      - FederalTIN
      - RateEffectiveDate
      - RateExpirationDate
      - PlanId
      - RatingAreaId
      - Tobacco
      - Age
      - IndividualRate
      - IndividualTobaccoRate
      - Couple
      - PrimarySubscriberAndOneDependent
      - PrimarySubscriberAndTwoDependents
      - PrimarySubscriberAndThreeOrMoreDependents
      - CoupleAndOneDependent
      - CoupleAndTwoDependents
      - CoupleAndThreeOrMoreDependents
      - RowNumber


Overwriting store.yaml


In [15]:
import yaml


In [16]:
import utility as util
import yaml

config_data = util.read_config_file("/content/store.yaml")


In [17]:
#data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'Rate',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate - IssuerId2 - FederalTIN - RateEffectiveDate - RateExpirationDate - PlanId - RatingAreaId - Tobacco - Age - IndividualRate - IndividualTobaccoRate - Couple - PrimarySubscriberAndOneDependent - PrimarySubscriberAndTwoDependents - PrimarySubscriberAndThreeOrMoreDependents - CoupleAndOneDependent - CoupleAndTwoDependents - CoupleAndThreeOrMoreDependents - RowNumber']}

In [18]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv('/content/Rate.csv',delimiter=',')
df_sample.head()


,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,FederalTIN,RateEffectiveDate,RateExpirationDate,...,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,RowNumber
0,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,29.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
1,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,14
2,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,15
3,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
4,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16


In [19]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "/content/" + config_data['file_name'] + f'.{file_type}'

In [20]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

<ipython-input-20-cf35396d3a9f>:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])


,BusinessYear,StateCode,IssuerId,SourceName,VersionNum,ImportDate,IssuerId2,FederalTIN,RateEffectiveDate,RateExpirationDate,...,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents,RowNumber
0,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,29.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0
1,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,14.0
2,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,36.95,NaN,73.9,107.61,107.61,107.61,144.56,144.56,144.56,15.0
3,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
4,2014,AK,21989,HIOS,6,2014-03-19 07:06:49,21989,93-0438772,2014-01-01,2014-12-31,...,32.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0


In [22]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['businessyear', 'statecode', 'issuerid', 'sourcename', 'versionnum',
       'importdate', 'issuerid2', 'federaltin', 'rateeffectivedate',
       'rateexpirationdate', 'planid', 'ratingareaid', 'tobacco', 'age',
       'individualrate', 'individualtobaccorate', 'couple',
       'primarysubscriberandonedependent', 'primarysubscriberandtwodependents',
       'primarysubscriberandthreeormoredependents', 'coupleandonedependent',
       'coupleandtwodependents', 'coupleandthreeormoredependents',
       'rownumber'],
      dtype='object')
columns of YAML are: ['BusinessYear - StateCode - IssuerId - SourceName - VersionNum - ImportDate - IssuerId2 - FederalTIN - RateEffectiveDate - RateExpirationDate - PlanId - RatingAreaId - Tobacco - Age - IndividualRate - IndividualTobaccoRate - Couple - PrimarySubscriberAndOneDependent - PrimarySubscriberAndTwoDependents - PrimarySubscriberAndThreeOrMoreDependents - CoupleAndOneDependent - CoupleAndTwoDependents - CoupleAndThre

In [23]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('/content/Rate.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('Rate.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:788: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:788: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:788: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:788: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:788: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  df.to_csv(f, **kwargs)
/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:788: 

['/content/Rate.csv.gz/0.part',
 '/content/Rate.csv.gz/1.part',
 '/content/Rate.csv.gz/2.part',
 '/content/Rate.csv.gz/3.part',
 '/content/Rate.csv.gz/4.part',
 '/content/Rate.csv.gz/5.part',
 '/content/Rate.csv.gz/6.part',
 '/content/Rate.csv.gz/7.part',
 '/content/Rate.csv.gz/8.part']

In [25]:
#number of files in gz format folder
import os
entries = os.listdir('/content/Rate.csv.gz')
for entry in entries:
    print(entry)

6.part
7.part
3.part
5.part
2.part
1.part
8.part
0.part
4.part


In [26]:
#size of the gz format folder
os.path.getsize('/content/Rate.csv.gz')

4096